<a href="https://colab.research.google.com/github/Mohammed-Taasir/CSE508_Winter2023_A2_14/blob/main/A2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/IR Assignment-1/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [3]:
path = '/content/CSE508_Winter2023_Dataset/'

In [67]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle
import nltk
import random
import math
import heapq
import operator
from collections import Counter
from tqdm import tqdm

In [53]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Defining Helper Functions

### Read from files

In [7]:
  def getListOfFiles(directory):

    # Parameters: directory: type(string)        
    # returns: list of all files in directory with the full path of file
    
    list_of_files = []

    for i in range(len(filenames)):
      fi = "/content/CSE508_Winter2023_Dataset/"+filenames[i]
      list_of_files.append(fi)
    
    return list_of_files

### Preprocessing Functions

In [8]:
def lowercase(data):

    # Parameters: data: type(string)
    # returns: lowercase of data   
     
    return data.lower()

In [22]:
def perform_word_tokenize(corpus):
  
    # Parameters:corpus: type(string)   
    # returns word-level tokenization of corpus

    return word_tokenize(corpus)

In [9]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
  
    # Parameters: tokens: type(list)
    #             stopwords_set: type(set)
    # returns: tokens without stopwords

    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set] 
    return tokens_sans_stopwords

In [10]:
def remove_punctuation_from_tokens(tokens):

    # Parameters: tokens: type(list)
    # returns: tokens without punctuation

    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    return tokens_sans_punctuation

In [11]:
def remove_blank_space_tokens(tokens):
    
    #Parameters: tokens: type(list)
    #returns: tokens without blank tokens

    tokens_sans_blank_space = [x for x in tokens if x!='']  
    return tokens_sans_blank_space

In [44]:
def preprocess(corpus, stopwords_set):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    #print(len(lowercase_corpus))
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    #print(len(word_tokens))
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    #print(len(word_tokens_sans_stopwords))
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    #print(len(word_tokens_sans_punctuation))
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
    #print(len(word_tokens_sans_blank_tokens))
    
    # Stem tokens
    #word_tokens_final = stemming(word_tokens_sans_blank_tokens)
    
    return word_tokens_sans_blank_tokens

In [45]:
def preprocess_documents(list_of_files, stopwords_set):
    '''
    Parameters:
        list_of_files: type(list)
        stopwords_set: type(set)
    
    returns: list of tokens obtained by preprocessing documents in all classes
    '''
    preprocessed_list_of_docs_tokens = []
    for doc_path in list_of_files:
        file = open(doc_path, 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set)
        preprocessed_list_of_docs_tokens.append(doc_tokens)
    
    pi_file = open('Q1_tf_idf.pkl', 'wb')
    pickle.dump(preprocessed_list_of_docs_tokens, pi_file)
    pi_file.close()
    
    return preprocessed_list_of_docs_tokens

In [13]:
def create_file_dictionary(list_of_files):
    
    # Paramteres: list_of_files: type(string)
    # returns: file_dictionary with integer key and path_of_file as value
  
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

# Q1. Data Preprocessing

## (i) Relevant Text Extraction

In [14]:
corpus_root='/content/CSE508_Winter2023_Dataset/'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [70]:
# Printing names of all files

filenames=corpus.fileids()
# print(filenames)

In [16]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


## First 5 Documents

In [17]:
# Printing contents of first 5 files before extracting

for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the des

## Extracting content between TITLE and TEXT tag from all documents.

In [18]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

## First 5 Documents after extracting contents between TITLE and TEXT tag.

In [19]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-----------------------------------------------

# (ii) Preprocessing

## 1. Lowercase the text

In [20]:
newfiles=[]
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-------------------------------------------------

## 2. Perform tokenization

In [23]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  newfiles[i] = filedata
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'with', 'supporting', 'evidence', ',', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control'

In [24]:
stopwords_set = set(stopwords.words('english'))

## 3. Remove stopwords

In [25]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  newfiles[i] = tokens_sans_stopwords
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '.', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '.', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '.', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'supporting', 'evidence', ',', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', '/destalling/', 'boundary-layer-control', 'effect', '.', 'integrated', 'remaining', 'lift', 'increment', ',', 'subtracting', 'destalling', 'lift', ',', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '.', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '.']
--

## 4. Remove punctuations

In [26]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_punctuation = remove_punctuation_from_tokens(filedata)
  newfiles[i] = tokens_sans_punctuation
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '', 'comparative', 'span', 'loading', 'curves', '', 'together', 'supporting', 'evidence', '', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', '', 'integrated', 'remaining', 'lift', 'increment', '', 'subtracting', 'destalling', 'lift', '', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '']
----------------

## 5. Remove blank space tokens

In [27]:
lent = 0
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_blank_space = remove_blank_space_tokens(filedata)
  newfiles[i] = tokens_sans_blank_space
  lent += len(newfiles[i])
print(len(newfiles))
print(lent)

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
127377
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', 'comparative', 'span', 'loading', 'curves', 'together', 'supporting', 'evidence', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', 'integrated', 'remaining', 'lift', 'increment', 'subtracting', 'destalling', 'lift', 'found', 'agree', 'well', 'potential', 'flow', 'theory', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment']
-------------------------------------------------

In [28]:
fileData = newfiles

# TF-IDF Matrix [25 points]

Defining helper functions

In [31]:
def create_file_dictionary(list_of_files):
    '''
    Paramteres:
        list_of_files: type(string)
    
    returns: file_dictionary with integer key and path_of_file as value
    '''
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

In [32]:
def create_2_way_mapping_of_term_and_doc(global_vocabulary):
    '''
    Parameter:
        global_vocabulary: type(list)
    
    returns: 2 dictionaries with key-value pair as term:docID and docID:term respectively
    '''
    term_vs_ID_dict = {}
    ID_vs_term_dict = {}
    
    for idx,term in enumerate(global_vocabulary):
        ID_vs_term_dict[idx] = term
        term_vs_ID_dict[term] = idx
    
    return term_vs_ID_dict, ID_vs_term_dict

In [33]:
def get_total_words_in_doc(TF_IDF_doc_vs_term_matrix):
    return np.sum(TF_IDF_doc_vs_term_matrix, axis=1)

In [34]:
def get_max_freq_term_in_doc(matrix):
    return np.amax(matrix, axis=1)

In [35]:
def calculate_IDF(inverted_index, total_documents):
    IDF = {}
    for term in inverted_index:
        IDF[term] = np.log10(total_documents/(1 + len(inverted_index[term])))
    return IDF

In [36]:
def Binary_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: binary weighting scheme
    '''
    return np.where((TF_IDF_doc_vs_term_matrix <= 0),0,1).astype('float')

In [37]:
def Raw_count_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: raw count weighing scheme
    '''
    return TF_IDF_doc_vs_term_matrix

In [38]:
def Raw_count_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: raw count weighing scheme
    '''
    return TF_IDF_doc_vs_term_matrix

In [48]:
def Term_frequency_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix, total_words_in_doc):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
        total_words_in_doc: type(np.array)
    
    returns: term frequency weighing scheme
    '''
    return (TF_IDF_doc_vs_term_matrix.T / total_words_in_doc).T

In [39]:
def Log_Normalization_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: log normalization weighing scheme
    '''
    return np.log10(1+TF_IDF_doc_vs_term_matrix)

In [40]:
def Double_Normalization_weighing_TF_IDF(TF_IDF_doc_vs_term_matrix, max_freq_term_in_doc):
    '''
    Parameters:
        TF_IDF_doc_vs_term_matrix: type(np.ndarray)
    
    returns: double normalization weighing scheme
    '''
    matrix = (TF_IDF_doc_vs_term_matrix.T / max_freq_term_in_doc).T

    return 0.5 + 0.5*matrix

In [41]:
def compute_variants(matrix, total_terms, max_freq, size_of_global_vocabulary, IDF, ID_vs_term_dict, isQuery):
    '''
    returns: variants of TF-IDF
    '''
    Binary = Binary_weighing_TF_IDF(matrix)
    Raw_count = Raw_count_weighing_TF_IDF(matrix)
    Term_frequency = Term_frequency_weighing_TF_IDF(matrix, total_terms)
    Log_Normalization = Log_Normalization_weighing_TF_IDF(matrix)
    Double_Normalization = Double_Normalization_weighing_TF_IDF(matrix, max_freq)
    
    if(isQuery):
        for i in tqdm(range(size_of_global_vocabulary)):
            IDF_factor = IDF[ID_vs_term_dict[i]]
            Binary[i] *= IDF_factor
            Raw_count[i] *= IDF_factor
            Term_frequency[i] *= IDF_factor
            Log_Normalization[i] *= IDF_factor
            Double_Normalization[i] *= IDF_factor
    else:
        for i in tqdm(range(size_of_global_vocabulary)):
            IDF_factor = IDF[ID_vs_term_dict[i]]
            Binary[:,i] *= IDF_factor
            Raw_count[:,i] *= IDF_factor
            Term_frequency[:,i] *= IDF_factor
            Log_Normalization[:, i] *= IDF_factor
            Double_Normalization[:, i] *= IDF_factor
    
    return Binary, Raw_count, Term_frequency, Log_Normalization, Double_Normalization

In [42]:
def topN(N, TF_IDF, query_of_same_variant, file_dictionary):
    '''
    evaluates and displays top N relevant documents based on score
    '''
    tf_idf_score = np.dot(TF_IDF, query_of_same_variant)
    tf_idf_score = {file_dictionary[i]:tf_idf_score[i] for i in range(len(tf_idf_score))}
    
    relevant_docs = list(sorted(tf_idf_score.items(), key=operator.itemgetter(1),reverse=True))[:N]
    for docs in relevant_docs:
        print('Score: {}  Document: {}'.format(docs[1], docs[0]))
    print()

MAIN

In [50]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    total_documents = len(file_dictionary)
    
    # preprocessed_list_of_docs_tokens = preprocess_documents(list_of_files, stopwords_set)
    pi_file = open('Q1_tf_idf.pkl', 'rb')
    preprocessed_list_of_docs_tokens = pickle.load(pi_file)
    pi_file.close()
    
    # Global list of terms
    global_list_of_terms = []
    for doc in preprocessed_list_of_docs_tokens:
        global_list_of_terms.extend(doc)
    
    # finding all distinct terms across all documents
    global_vocabulary = list(set(global_list_of_terms))
    size_of_global_vocabulary = len(global_vocabulary)
    
    # term vs docID 2 way mapping
    term_vs_ID_dict, ID_vs_term_dict = create_2_way_mapping_of_term_and_doc(global_vocabulary)
    
    # term-doc matrix (for storing frequency of each word of global_vocabulary in doc)
    inverted_index = {}
    size_of_TF_IDF_matrix = (total_documents, size_of_global_vocabulary)
    TF_IDF_doc_vs_term_matrix = np.zeros(size_of_TF_IDF_matrix, dtype=float)
    
    for i in tqdm(range(len(preprocessed_list_of_docs_tokens))):
        for term in preprocessed_list_of_docs_tokens[i]:
            if(term in inverted_index):
                if(inverted_index[term][-1]!=i):
                    inverted_index[term].append(i)
            else:
                inverted_index[term] = [i]
            TF_IDF_doc_vs_term_matrix[i][term_vs_ID_dict[term]] += 1
    
    total_words_in_doc = get_total_words_in_doc(TF_IDF_doc_vs_term_matrix)
    
    max_freq_term_in_doc = get_max_freq_term_in_doc(TF_IDF_doc_vs_term_matrix)
    
    # Calculate IDF
    IDF = calculate_IDF(inverted_index, total_documents)
    
    Binary_tf_idf, Raw_count_tf_idf, Term_frequency_tf_idf, Log_Normalization_tf_idf, Double_Normalization_tf_idf = compute_variants(TF_IDF_doc_vs_term_matrix, total_words_in_doc, max_freq_term_in_doc, size_of_global_vocabulary, IDF, ID_vs_term_dict, False)
    
    query = input("Input query: ")
    sanitized_query = preprocess(query, stopwords_set)
    print("Sanitized query: ", sanitized_query)

    query_frequency = {}
    for query_token in sanitized_query:
        if query_token in query_frequency:
            query_frequency[query_token]+=1
        else:
            query_frequency[query_token]=1
    
    # create query frequency vector
    query_frequency_vector = np.zeros((size_of_global_vocabulary,1))
    for token in query_frequency.keys():
        if token in global_vocabulary:
            query_frequency_vector[term_vs_ID_dict[token]] = query_frequency[token]
    
    
    max_freq_token = query_frequency[max(query_frequency, key=query_frequency.get)]
    query_binary, query_raw_count, query_term_frequency, query_log_normalization, query_double_normalization = compute_variants(query_frequency_vector, len(sanitized_query), max_freq_token, size_of_global_vocabulary, IDF, ID_vs_term_dict, True)
    
    print('Binary Scheme: Top 5 relevant documents are:')
    topN(5, Binary_tf_idf, query_binary, file_dictionary)
    
    print('Raw Count Scheme: Top 5 relevant documents are:')
    topN(5, Raw_count_tf_idf, query_raw_count, file_dictionary)
    
    print('Term Frequency Scheme: Top 5 relevant documents are:')
    topN(5, Term_frequency_tf_idf, query_term_frequency, file_dictionary)
    
    print('Log Normalization Scheme: Top 5 relevant documents are:')
    topN(5, Log_Normalization_tf_idf, query_log_normalization, file_dictionary)
    
    print('Double_Normalization Scheme: Top 5 relevant documents are:')
    topN(5, Double_Normalization_tf_idf, query_double_normalization, file_dictionary)

In [54]:
if __name__ == "__main__":
    main()

100%|██████████| 8996/8996 [00:00<00:00, 13718.62it/s]


Input query: study
Sanitized query:  ['study']


100%|██████████| 8996/8996 [00:00<00:00, 78218.95it/s]

Binary Scheme: Top 5 relevant documents are:
Score: [0.9938274]  Document: /content/CSE508_Winter2023_Dataset/cranfield0001
Score: [0.9938274]  Document: /content/CSE508_Winter2023_Dataset/cranfield0002
Score: [0.9938274]  Document: /content/CSE508_Winter2023_Dataset/cranfield0008
Score: [0.9938274]  Document: /content/CSE508_Winter2023_Dataset/cranfield0011
Score: [0.9938274]  Document: /content/CSE508_Winter2023_Dataset/cranfield0017

Raw Count Scheme: Top 5 relevant documents are:
Score: [2.9814822]  Document: /content/CSE508_Winter2023_Dataset/cranfield0421
Score: [2.9814822]  Document: /content/CSE508_Winter2023_Dataset/cranfield0907
Score: [2.9814822]  Document: /content/CSE508_Winter2023_Dataset/cranfield0991
Score: [1.9876548]  Document: /content/CSE508_Winter2023_Dataset/cranfield0002
Score: [1.9876548]  Document: /content/CSE508_Winter2023_Dataset/cranfield0056

Term Frequency Scheme: Top 5 relevant documents are:
Score: [0.06211421]  Document: /content/CSE508_Winter2023_Data

Score: [14671.00742777]  Document: /content/CSE508_Winter2023_Dataset/cranfield0718
Score: [14658.84278181]  Document: /content/CSE508_Winter2023_Dataset/cranfield0212
Score: [14654.71266031]  Document: /content/CSE508_Winter2023_Dataset/cranfield1167
Score: [14654.42523678]  Document: /content/CSE508_Winter2023_Dataset/cranfield0344
Score: [14653.87756631]  Document: /content/CSE508_Winter2023_Dataset/cranfield1088



#Jaccard Coefficient [15 marks]

Union function

In [55]:
def union(l1,l2):
    union_list = list(set(l1) | set(l2))
    return union_list

Intersection function

In [56]:
def intersection(l1,l2):
    intersection_list = list(set(l1) & set(l2))
    return intersection_list

Calculate Jaccard coefficient

In [62]:
def jaccardCoefficient(query, list_of_files):
    for i,filePath in enumerate (list_of_files):
        file = open(filePath, encoding="utf8", errors = "ignore")
        read = file.read()    
        file.close()
    
        sanitized_query = preprocess(read, stopwords_set)                    
        # calculate jaccard coefficient value based on formula that is intersection of document and query divided by union of 
        # document and query
        jaccard_coefficient[filePath] = len(intersection(sanitized_query, query))/len(union(sanitized_query, query))

In [60]:
len(stopwords_set)

179

Finding top 10 relevant documents

In [65]:
# Get List of Files in Dataset
list_of_files = getListOfFiles("/content/CSE508_Winter2023_Dataset/")
#print(list_of_files)
jaccard_coefficient = {}        
sentence_query = input("Enter the query: ")
query = preprocess(sentence_query, stopwords_set)      # Query Processing
jaccardCoefficient(query, list_of_files)
# it counts the elements in the dictionary and prints the 5 most common documents based on it.
#k = Counter(dict(jaccard_coefficient)).most_common(5) 
print(" Top 5 relevant documents based on Jaccard Coefficient ")  
for i in Counter(dict(jaccard_coefficient)).most_common(10) :
    print("{} --> {}".format(i[1],i[0]))

Enter the query: supersonic wind tunnel
 Top 5 relevant documents based on Jaccard Coefficient 
0.10714285714285714 --> /content/CSE508_Winter2023_Dataset/cranfield0430
0.08695652173913043 --> /content/CSE508_Winter2023_Dataset/cranfield0429
0.07692307692307693 --> /content/CSE508_Winter2023_Dataset/cranfield1142
0.07407407407407407 --> /content/CSE508_Winter2023_Dataset/cranfield1306
0.06976744186046512 --> /content/CSE508_Winter2023_Dataset/cranfield0969
0.06896551724137931 --> /content/CSE508_Winter2023_Dataset/cranfield0594
0.06666666666666667 --> /content/CSE508_Winter2023_Dataset/cranfield0598
0.06666666666666667 --> /content/CSE508_Winter2023_Dataset/cranfield0795
0.06666666666666667 --> /content/CSE508_Winter2023_Dataset/cranfield1243
0.06060606060606061 --> /content/CSE508_Winter2023_Dataset/cranfield1305


In [66]:
# queries -> jet propulsion, supersonic wind tunnel